<a href="https://colab.research.google.com/github/mr-44-bs/createLyrics/blob/main/B_Finetuning2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') #googledriveと接続

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install mlflow #実行環境、モデル、パラメータ、評価指標などの実験管理を行ってくれるプラットフォームであるmlflowのインストール

In [ ]:
!pip install transformers #transformersのインストール

In [ ]:
!pip install fugashi #fugashiのインストール

In [ ]:
!pip install ipadic #ipadicのインストール

In [ ]:
!pip install pytorch-lightning #pytorch-lightningのインストール

In [ ]:
import random
import glob
from tqdm import tqdm

import torch
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from mlflow import log_metric, log_param, log_artifact
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

#環境を整えている

In [ ]:
df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/lyrics/MrChildren_train.csv", sep=',', header=0,
                 names=["title", "lyrics"])

df["label"] = 0

In [ ]:
df

In [ ]:
df1 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/lyrics/spitz_train.csv", sep=',', header=0,
                 names=["title", "lyrics"])

df1["label"] = 1

In [ ]:
df1

In [ ]:
df2 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/lyrics/Ringo_train.csv", sep=',', header=0,
                 names=["title", "lyrics"])

df2["label"] = 2

In [ ]:
df2

In [ ]:
df3 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/lyrics/back_train.csv", sep=',', header=0,
                 names=["title", "lyrics"])

df3["label"] = 3

In [ ]:
df3

In [ ]:
df4 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/lyrics/Rad_train.csv", sep=',', header=0,
                 names=["title", "lyrics"])

df4["label"] = 4

In [ ]:
df4

In [ ]:
df5 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/lyrics/B'z_train.csv", sep=',', header=0,
                 names=["title", "lyrics"])

df5["label"] = 5

In [ ]:
df_concat = pd.concat([df,df1,df2,df3,df4,df5],axis=0)
print(df_concat)

In [ ]:
labels = df_concat.label.values
lyrics = df_concat.lyrics.values
print(labels)
print(lyrics)

In [ ]:
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [ ]:
train_size = int(0.8 * len(df_concat))
val_size = len(df_concat) - train_size
test_size = int(val_size/2)
val_size2 = val_size-test_size

# データセットを分割
train, val = random_split(df_concat, [train_size, val_size])
test, val = random_split(val, [test_size, val_size2])

In [ ]:
max_length = 512

In [ ]:
dataset_for_loader = []

end_term = "[SEP]"

# 1文づつ処理
for x , label in zip(lyrics, labels):
    #sent= x  + end_term + y

    encoding = tokenizer(
            x,
            max_length=max_length,
            padding='max_length',
            truncation=True
        )

    encoding['labels'] = label # ラベルを追加
    encoding = { k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)


In [ ]:
train_size1 = int(0.8 * len(dataset_for_loader))
val_size1 = len(dataset_for_loader) - train_size
test_size1 = int(val_size/2)
val_size2 = val_size-test_size

# データセットを分割
train_dataset, val_dataset = random_split(dataset_for_loader, [train_size1, val_size1])
test_dataset, val_dataset = random_split(val_dataset, [test_size1, val_size2])

In [ ]:
dataloader_train = DataLoader(
    train_dataset, batch_size=16, shuffle=True
)
dataloader_val = DataLoader(val_dataset, batch_size=16)
dataloader_test = DataLoader(test_dataset, batch_size=1)

In [ ]:
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
bert_sc = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
bert_sc = bert_sc.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [ ]:
class BertForSequenceClassification_pl(pl.LightningModule):

    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        self.save_hyperparameters()

        #BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels = num_labels
        )

    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss) #損失関数 lossをaccuracyに変えることもできる
        return loss

    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch) #batchは読み込ませるデータの塊　ハイパーパラメータだから変えて比較するのもあり
        val_loss = output.loss
        self.log('val_loss', val_loss)

    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = (labels_predicted == labels).sum().item()
        accuracy = num_correct / labels.size(0)
        self.log('accuracy', accuracy)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)

In [ ]:
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor = 'val_loss',
    mode = 'min',#今回は損失関数で評価していて、損失関数が小さいものを採用する（accuracyの場合などはmax使ったりする）
    save_top_k = 1,#一番損失関数が小さいモデルを保存する
    save_weights_only = True,
    dirpath  = 'model/'
)

early_stopping = pl.callbacks.EarlyStopping( #学習が収束したあたりで学習を終わらせる
    monitor = 'val_loss',
    mode = 'min',
    patience = 10
)

trainer = pl.Trainer(
    accelerator = 'gpu',
    #devices = 2,
    #gpus = [2],
    max_epochs = 20,#自分が用意したデータを何週学習させるか指定
    callbacks = [checkpoint, early_stopping]
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
model = BertForSequenceClassification_pl(
    MODEL_NAME, num_labels=6, lr=2e-5
)

trainer.fit(model, dataloader_train, dataloader_val)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
best_model_path = checkpoint.best_model_path # ベストモデルのファイル
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)

In [ ]:
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl.load_from_checkpoint(
    best_model_path
)

# Transformers対応のモデルを./model_transformesに保存
model.bert_sc.save_pretrained('./model_transformers')

In [ ]:
bert_sc = BertForSequenceClassification.from_pretrained(
    './model_transformers'
)#ベストモデルを保存したディレクトリからBertForSequenceClassificationモデルを読み込む

bert_sc.cuda()#GPU上で実行するためにモデルをGPUメモリに転送

In [ ]:
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

In [ ]:
df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/lyrics/MrChildren_test.csv", sep=',', header=0,
                 names=["title", "lyrics"])

df["label"] = 0

In [ ]:
df1 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/lyrics/spitz_test.csv", sep=',', header=0,
                 names=["title", "lyrics"])

df1["label"] = 1

In [ ]:
df2 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/lyrics/Ringo_test.csv", sep=',', header=0,
                 names=["title", "lyrics"])

df2["label"] = 2

In [ ]:
df3 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/lyrics/back_test.csv", sep=',', header=0,
                 names=["title", "lyrics"])

df3["label"] = 3

In [ ]:
df4 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/lyrics/Rad_test.csv", sep=',', header=0,
                 names=["title", "lyrics"])

df4["label"] = 4

In [ ]:
df5 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/lyrics/B'z_test.csv", sep=',', header=0,
                 names=["title", "lyrics"])

df5["label"] = 5

In [ ]:
df2_concat = pd.concat([df,df1,df2,df3,df4,df5],axis=0)
print(df2_concat)

In [ ]:
labels = df2_concat.label.values
lyrics = df2_concat.lyrics.values
print(labels)
print(lyrics)

In [ ]:
predicted = []
correct_labels = []
wrong = []
i = 0

for x , label in zip(lyrics, labels):

    correct_labels.append(label)
    correct = label

    encoding = tokenizer(
            x,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

    encoding = { k: v.cuda() for k, v in encoding.items() }

    with torch.no_grad():
      output = bert_sc.forward(**encoding)
      scores = output.logits
      labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist()
      predicted.append(labels_predicted)

    i += 1
'''
    if labels_predicted == correct:
        wrong.append(i)
'''


In [ ]:
test_num = len(predicted)
num_correct = 0
tp = 0
fp = 0
fn = 0
tn = 0

print(predicted)
print(correct_labels)

for i in range(test_num):
    if predicted[i] == correct_labels[i]:
        num_correct += 1
'''
    if predicted[i] == 0 and correct_labels[i] == 0:
        tp += 1

    if (predicted[i] == 1 and correct_labels[i] == 0) or (predicted[i] == 1 and correct_labels[i] == 2):
        fp += 1

    if (predicted[i] == 0 and correct_labels[i] == 1) or (predicted[i] == 2 and correct_labels[i] == 1):
        fn += 1

    if (predicted[i] == 0 and correct_labels[i] == 1) or (predicted[i] == 2 and correct_labels[i] == 1):
        tn += 1
'''
accuracy = num_correct / test_num
#recall = tp /(tp + fn)
#precision = tp /(tp + fp)
#f_value = 2*recall*precision / (precision + recall)
print("accuracy: " + str(accuracy))
# print("recall: " + str(recall)) #先輩の3値分類の計算方法だから色々計算方法適切なものに変える
# print("precision: " + str(precision))
# print("f_value: " + str(f_value))

In [ ]:
lyrics = []
labels = []

In [ ]:
with open('/content/gdrive/My Drive/Colab Notebooks/lyrics/M_form.txt') as f:
  for line in f:
     if len(line) > 20:
        line = line.replace('☆', '')
        line = line.replace('★', '')
        lyrics.append(line)
        labels.append(0)
        #print (line)


In [ ]:
with open('/content/gdrive/My Drive/Colab Notebooks/lyrics/spitz_form.txt') as f:
  for line in f:
     if len(line) > 20:
        line = line.replace('☆', '')
        line = line.replace('★', '')
        lyrics.append(line)
        labels.append(1)
        #print (line)


In [ ]:

with open('/content/gdrive/My Drive/Colab Notebooks/lyrics/Ringo_form.txt') as f:
  for line in f:
     if len(line) > 20:
        line = line.replace('☆', '')
        line = line.replace('★', '')
        lyrics.append(line)
        labels.append(2)
        #print (line)


In [ ]:

with open('/content/gdrive/My Drive/Colab Notebooks/lyrics/back_form.txt') as f:
  for line in f:
     if len(line) > 20:
        line = line.replace('☆', '')
        line = line.replace('★', '')
        lyrics.append(line)
        labels.append(3)
        #print (line)


In [ ]:
with open('/content/gdrive/My Drive/Colab Notebooks/lyrics/Rad_form.txt') as f:
  for line in f:
     if len(line) > 20:
        line = line.replace('☆', '')
        line = line.replace('★', '')
        lyrics.append(line)
        labels.append(4)


In [ ]:
with open('/content/gdrive/My Drive/Colab Notebooks/lyrics/Bz_form.txt') as f:
  for line in f:
     if len(line) > 20:
        line = line.replace('☆', '')
        line = line.replace('★', '')
        lyrics.append(line)
        labels.append(5)


In [ ]:
print(lyrics)
print(labels)

In [ ]:
predicted = []
correct_labels = []
wrong = []
i = 0

for x , label in zip(lyrics, labels):

    correct_labels.append(label)
    correct = label

    encoding = tokenizer(
            x,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

    encoding = { k: v.cuda() for k, v in encoding.items() }

    with torch.no_grad():
      output = bert_sc.forward(**encoding)
      scores = output.logits
      labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist()
      predicted.append(labels_predicted)

    i += 1
'''
    if labels_predicted == correct:
        wrong.append(i)
'''


In [ ]:
test_num = len(predicted)
num_correct = 0
tp = 0
fp = 0
fn = 0
tn = 0

print(predicted)
print(correct_labels)

for i in range(test_num):
    if predicted[i] == correct_labels[i]:
        num_correct += 1

    if predicted[i] == correct_labels[i]:
        tp += 1

    if predicted[i] != correct_labels[i]:
        fp += 1

    if predicted[i] != correct_labels[i]:
        fn += 1

    #if (predicted[i] == 0 and correct_labels[i] == 1) or (predicted[i] == 2 and correct_labels[i] == 1):
        #fn += 1

    if (predicted[i] == 0 and correct_labels[i] == 1) or (predicted[i] == 2 and correct_labels[i] == 1):
        tn += 1

accuracy = num_correct / test_num
#recall = tp /(tp + fn)
#precision = tp /(tp + fp)
#f_value = 2*recall*precision / (precision + recall)
print("accuracy: " + str(accuracy))
#print("recall: " + str(recall)) #先輩の3値分類の計算方法だから色々計算方法適切なものに変える
# print("precision: " + str(precision))
# print("f_value: " + str(f_value))